In [1]:
import sys
reload(sys)
sys.setdefaultencoding('utf-8')

In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('../data/huizhou-final-v2-seg.csv')

In [6]:
df.info()

In [7]:
df.head()

,url,domain,title,content,content_seg
0,http://cul.anhuinews.com/system/2016/11/28/007...,anhuinews.com,洪建华夫妇的竹刻人生,谁能想到，洪建华夫妇这对一无学历、二无家传、三无师承的“三无大师”成了省级非物质文化遗产·徽...,谁 能 想到 ， 洪 建华 夫妇 这 对 一 无 学历 、 二 无 家传 、 三无 师承 的...
1,http://cul.anhuinews.com/system/2016/11/21/007...,anhuinews.com,徽州民歌传人操明花校园授艺,近日，省非物质文化遗产徽州民歌传承人操明花前往休宁县海阳二小、歙县城关小学和徽州区岩寺镇中心...,近日 ， 省非 物质 文化遗产 徽州 民歌 传承 人操 明花 前往 休宁县 海阳 二小 、 ...
2,http://cul.anhuinews.com/system/2016/11/03/007...,anhuinews.com,黄山楹联漫谈,黄山楹联第一人当推明朝诗人余绍祉。他遍历祖国名山大川，见到无数奇峰怪石，却感到抵不上黄山峰林...,黄山 楹联 第一 人当 推 明朝 诗人 余绍祉 。 他 遍历 祖国 名山大川 ， 见到 无数...
3,http://cul.anhuinews.com/system/2016/01/26/007...,anhuinews.com,徽派雕刻——徽州木雕艺术,徽州民居、家具木板和圆木雕刻艺术的简称。木雕在旧属徽州各县分布之广在全国屈指可数，宅院内的屏...,徽州 民居 、 家具 木板 和 圆木 雕刻 艺术 的 简称 。 木雕 在 旧属 徽州 各县 ...
4,http://cul.anhuinews.com/system/2016/01/26/007...,anhuinews.com,徽派雕刻——徽州砖雕艺术,徽州砖雕徽州砖雕的造型，来源于汉画象砖。汉画象砖大都是模印砖坯刻画后入密烧制，嵌窃时再进行修...,徽州 砖雕 徽州 砖雕 的 造型 ， 来源于 汉 画象 砖 。 汉 画象 砖 大都 是 模印...


In [8]:
df = df[df['domain'] != 'huaxia.com']

In [11]:
from __future__ import absolute_import
from __future__ import division, print_function, unicode_literals

from sumy.parsers.html import HtmlParser
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lsa import LsaSummarizer as Summarizer
from sumy.nlp.stemmers import Stemmer
from sumy.utils import get_stop_words

In [12]:
from sumy.parsers.plaintext import PlaintextParser

In [13]:
LANGUAGE = "english"
SENTENCES_COUNT = 5

In [5]:
url='http://en.wikipedia.org/wiki/Automatic_summarization'
parser = HtmlParser.from_url(url, Tokenizer(LANGUAGE))

In [9]:
msk1 = df['content'].str.contains('徽州') 
msk2 = df['content'].str.contains('徽文化') 
msk = msk1 | msk2
df_check = df[msk]

In [10]:
df_check.info()

In [ ]:
#Simple library and command line utility for extracting summary from HTML pages or plain texts. The package 
#also contains simple evaluation framework for text summaries. Implemented summarization methods:

#    Luhn - heurestic method, reference
#    Edmundson heurestic method with previous statistic research, reference
#    Latent Semantic Analysis, LSA - one of the algorithm from http://scholar.google.com/citations?user=0fTuW_YAAAAJ&hl=en I think the author is using more advanced algorithms now. Steinberger, J. a JeĹľek, K. Using latent semantic an and summary evaluation. In In Proceedings ISIM ‘04. 2004. S. 93-100.
#    LexRank - Unsupervised approach inspired by algorithms PageRank and HITS, reference
#    TextRank - some sort of combination of a few resources that I found on the internet. I really don’t remember the sources. Probably Wikipedia and some papers in 1st page of Google :)
#    SumBasic - Method that is often used as a baseline in the literature. Source: Read about SumBasic
#    KL-Sum - Method that greedily adds sentences to a summary so long as it decreases the KL Divergence. Source: Read about KL-Sum


In [14]:
from time import time
t0 = time()
for i in range(0, 100):  #8.591509103775024s
    print (i, '------------------')
    parser = HtmlParser.from_string(df_check.iloc[i]['content'], 'none', Tokenizer(LANGUAGE))
    stemmer = Stemmer(LANGUAGE)
    summarizer = Summarizer(stemmer)
    summarizer.stop_words = get_stop_words(LANGUAGE)
    #print (parser.document)
    idx = 0
    for sentence in summarizer(parser.document, SENTENCES_COUNT):
            if (1): #idx != 0):
                print(sentence)
                print ('*********')
            idx += 1
time() - t0

2.219939947128296

In [15]:
import codecs
from textrank4zh import TextRank4Keyword, TextRank4Sentence
tr4s = TextRank4Sentence()

In [16]:
t0 = time()
from time import time
t0 = time()
for i in range(0, 100):  #11.573723077774048
    print (i, '***********************')
    tr4s.analyze(text=df_check.iloc[i]['content'], lower=True, source = 'all_filters')
    for item in tr4s.get_key_sentences(num=3):
        print(item.index, item.weight, item.sentence)  # index是语句在文本中位置，weight是权重
        print ('------------------')
time() - t0

69.57102489471436

In [17]:
# encoding=utf-8  
import jieba  
import networkx as nx  
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer  
  
def cut_sentence(sentence):  
    """ 
    分句 
    :param sentence: 
    :return: 
    """  
    if not isinstance(sentence, unicode):  
        sentence = sentence.decode('utf-8')  
    delimiters = frozenset(u'。！？')  
    buf = []  
    for ch in sentence:  
        buf.append(ch)  
        if delimiters.__contains__(ch):  
            yield ''.join(buf)  
            buf = []  
    if buf:  
        yield ''.join(buf)  
  
  
def load_stopwords(path='/home/fhqplzj/data/orion/stopwords.txt'):  
    """ 
    加载停用词 
    :param path: 
    :return: 
    """  
    with open(path) as f:  
        stopwords = filter(lambda x: x, map(lambda x: x.strip().decode('utf-8'), f.readlines()))  
    stopwords.extend([' ', '\t', '\n'])  
    return frozenset(stopwords)  
  
def cut_words(sentence):  
    """ 
    分词 
    :param sentence: 
    :return: 
    """  
    stopwords = load_stopwords()  
    return filter(lambda x: not stopwords.__contains__(x), jieba.cut(sentence))  
  
def get_abstract(content, size=3):  
    """ 
    利用textrank提取摘要 
    :param content: 
    :param size: 
    :return: 
    """  
    docs = list(cut_sentence(content))  
    
    #tfidf_model = TfidfVectorizer(tokenizer=jieba.cut, stop_words=load_stopwords())  
    tfidf_model = TfidfVectorizer()  
    tfidf_matrix = tfidf_model.fit_transform(docs)  
    normalized_matrix = TfidfTransformer().fit_transform(tfidf_matrix)  
    similarity = nx.from_scipy_sparse_matrix(normalized_matrix * normalized_matrix.T)  
    scores = nx.pagerank(similarity)  
    tops = sorted(scores.iteritems(), key=lambda x: x[1], reverse=True)  
    size = min(size, len(docs))  
    indices = map(lambda x: x[0], tops)[:size]  
    return map(lambda idx: docs[idx], indices)  

In [49]:
s = u'要说现在当红的90后男星，那就不得不提鹿晗、吴亦凡、杨洋、张艺兴、黄子韬、陈学冬、刘昊然，2016年他们带来不少人气爆棚的影视剧。这些90后男星不仅有颜值、有才华，还够努力，2017年他们又有哪些傲娇的作品呢？到底谁会成为2017霸屏男神，让我们拭目以待吧。鹿晗2016年参演《盗墓笔记》、《长城》、《摆渡人》等多部电影，2017年他将重心转到了电视剧。他和古力娜扎主演的古装奇幻电视剧《择天记》将在湖南卫视暑期档播出，这是鹿晗个人的首部电视剧，也是其第一次出演古装题材。该剧改编自猫腻的同名网络小说，讲述在人妖魔共存的架空世界里，陈长生(鹿晗饰演)为了逆天改命，带着一纸婚书来到神都，结识了一群志同道合的小伙伴，在国教学院打开一片新天地。吴亦凡在2017年有更多的作品推出。周星驰监制、徐克执导的春节档《西游伏魔篇》，吴亦凡扮演“有史以来最帅的”唐僧。师徒四人在取经的路上，由互相对抗到同心合力，成为无坚不摧的驱魔团队。吴亦凡还和梁朝伟、唐嫣合作动作片《欧洲攻略》，该片讲述江湖排名第一、第二的林先生(梁朝伟饰)和王小姐(唐嫣饰)亦敌亦友，二人与助手乐奇(吴亦凡饰)分别追踪盗走“上帝之手”地震飞弹的苏菲，不想却引出了欧洲黑帮、美国CIA、欧盟打击犯罪联盟特工们的全力搜捕的故事。吴亦凡2017年在电影方面有更大突破，他加盟好莱坞大片《极限特工3：终极回归》，与范·迪塞尔、甄子丹、妮娜·杜波夫等一众大明星搭档，为电影献唱主题曲《JUICE》。此外，他还参演吕克·贝松执导的科幻电影《星际特工：千星之城》，该片讲述一个发生在未来28世纪星际警察穿越时空的故事，影片有望2017年上映。'  

In [19]:
t0  = time()
for i in range(0, 100):  
    print (i, '*****************')
    for t in get_abstract(df_check.iloc[i]['content']):  
        print (t)
        print ('--------------------')

time() - t0

0.9909038543701172

In [ ]:
# Text summarization with NLTK

In [20]:
from nltk.tokenize import sent_tokenize,word_tokenize
from nltk.corpus import stopwords
from collections import defaultdict
from string import punctuation
from heapq import nlargest

In [21]:
class FrequencySummarizer:
  def __init__(self, min_cut=0.1, max_cut=0.9):
    """
     Initilize the text summarizer.
     Words that have a frequency term lower than min_cut 
     or higer than max_cut will be ignored.
    """
    self._min_cut = min_cut
    self._max_cut = max_cut 
    self._stopwords = set(stopwords.words('english') + list(punctuation))

  def _compute_frequencies(self, word_sent):
    """ 
      Compute the frequency of each of word.
      Input: 
       word_sent, a list of sentences already tokenized.
      Output: 
       freq, a dictionary where freq[w] is the frequency of w.
    """
    freq = defaultdict(int)
    for s in word_sent:
      for word in s:
        if word not in self._stopwords:
          freq[word] += 1
    # frequencies normalization and fitering
    m = float(max(freq.values()))
    for w in freq.keys():
      freq[w] = freq[w]/m
      if freq[w] >= self._max_cut or freq[w] <= self._min_cut:
        del freq[w]
    return freq

  def summarize(self, text, n):
    """
      Return a list of n sentences 
      which represent the summary of text.
    """
    sents = sent_tokenize(text)
    assert n <= len(sents)
    word_sent = [word_tokenize(s.lower()) for s in sents]
    self._freq = self._compute_frequencies(word_sent)
    ranking = defaultdict(int)
    for i,sent in enumerate(word_sent):
      for w in sent:
        if w in self._freq:
          ranking[i] += self._freq[w]
    sents_idx = self._rank(ranking, n)    
    return [sents[j] for j in sents_idx]

  def _rank(self, ranking, n):
    """ return the first n sentences with highest ranking """
    return nlargest(n, ranking, key=ranking.get)



In [22]:
fs = FrequencySummarizer()

In [25]:
t0  = time()
for i in range(0, 100):  
    print (i, '*****************')
    for s in fs.summarize(df_check.iloc[i]['content'].decode('utf-8'), 2):
        print (s)
        print ('--------------------')

time() - t0

AssertionError: 